In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
data = "/content/drive/MyDrive/Colab Notebooks/Visual-Bert-Application/VQA_RAD Dataset Public.json"
df = pd.read_json(data)
df.drop(columns = ['qid' , 'phrase_type' ,'image_organ' , 'evaluation' , 'question_relation' , 'question_type' , 'answer_type', 'qid_linked_id' , 'image_case_url'] , inplace= True)
df

In [6]:
def question_rephrase(string:str):
    if string == 'NULL':
        return ''
    else:
        return string

In [7]:
df['question_rephrase'] = df['question_rephrase'].apply(question_rephrase)
df['question'] = df['question'] + " " + df['question_rephrase']
df.drop(columns = ['question_rephrase'] , inplace=True)

In [8]:
df['question_frame'] = df['question_frame'].apply(question_rephrase)
df['question'] = df['question']+" "+df['question_frame']
df.drop(columns=['question_frame'] , inplace=True)

In [9]:
df

,image_name,question,answer
0,synpic54610.jpg,Are regions of the brain infarcted?,Yes
1,synpic29265.jpg,Are the lungs normal appearing?,No
2,synpic29265.jpg,Is there evidence of a pneumothorax,No
3,synpic28602.jpg,What type of imaging does this not represent?,ultrasound
4,synpic29265.jpg,Is this a MRI of the chest?,no
...,...,...,...
2243,synpic56841.jpg,"Is this image modality T1, T2, or FLAIR? Is th...",FLAIR
2244,synpic26764.jpg,Is this a PA film? Is this film taken in a PA ...,Yes
2245,synpic26764.jpg,Is this film taken in a PA modality? Is this a...,Yes
2246,synpic31232.jpg,What is the hypodensity located inside the ver...,Nucleus Pulposus


In [10]:
def change_img_path(image_name:str):
    return f"VQA_RAD Image Folder/{image_name}"
df['image_name'] = df['image_name'].apply(change_img_path)

In [11]:
df['image_name']

,image_name
0,VQA_RAD Image Folder/synpic54610.jpg
1,VQA_RAD Image Folder/synpic29265.jpg
2,VQA_RAD Image Folder/synpic29265.jpg
3,VQA_RAD Image Folder/synpic28602.jpg
4,VQA_RAD Image Folder/synpic29265.jpg
...,...
2243,VQA_RAD Image Folder/synpic56841.jpg
2244,VQA_RAD Image Folder/synpic26764.jpg
2245,VQA_RAD Image Folder/synpic26764.jpg
2246,VQA_RAD Image Folder/synpic31232.jpg


In [19]:
n =len(df)
df = df[:int(0.5*n)]

In [20]:
df.shape

(1124, 3)

In [21]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [22]:
df.dropna(inplace=True)
print(df['question'].isna().sum())
print(df['question'])

<ipython-input-22-c64f9f573c18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [25]:
def pre_process(sentence:str):
    try:
        sen = nlp(sentence)
        tokens = [token.text.lower() for token in sen if not token.is_punct]
        re_sen = " ".join(tokens)
        re_sen.replace('?' , " ")
        return re_sen
    except:
        print(sentence)
        return str(sentence)

In [26]:
df['question'] = df['question'].apply(pre_process)

<ipython-input-26-dd59e1efc434>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['question'] = df['question'].apply(pre_process)


In [27]:
df

,image_name,question,answer
0,VQA_RAD Image Folder/synpic54610.jpg,are regions of the brain infarcted,Yes
1,VQA_RAD Image Folder/synpic29265.jpg,are the lungs normal appearing,No
2,VQA_RAD Image Folder/synpic29265.jpg,is there evidence of a pneumothorax,No
3,VQA_RAD Image Folder/synpic28602.jpg,what type of imaging does this not represent,ultrasound
4,VQA_RAD Image Folder/synpic29265.jpg,is this a mri of the chest,no
...,...,...,...
1119,VQA_RAD Image Folder/synpic33302.jpg,are there any consolidations present in the lu...,No
1120,VQA_RAD Image Folder/synpic33302.jpg,are there any pulmonary consolidations are the...,No
1121,VQA_RAD Image Folder/synpic16221.jpg,is this patient male or female what is the pat...,Female
1122,VQA_RAD Image Folder/synpic16221.jpg,what is the patient 's gender is this patient ...,Female


In [28]:
def lower_if (text:str):
    try:
        return text.lower()
    except:
        return text

In [30]:
df['answer'] = df['answer'].apply(lower_if)
df['answer'] = df['answer'].astype(str)

<ipython-input-30-e7c517e29ed7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = df['answer'].apply(lower_if)
<ipython-input-30-e7c517e29ed7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = df['answer'].astype(str)


In [32]:
non_string_answers = df[~df['answer'].apply(lambda x: isinstance(x, str))]
print(non_string_answers)
df.dropna(inplace=True)
print(df.dtypes)

Empty DataFrame
Columns: [image_name, question, answer]
Index: []
image_name    object
question      object
answer        object
dtype: object


<ipython-input-32-307ac6fa1eea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder # import the LabelEncoder

hot_enc = OneHotEncoder()
le = LabelEncoder() # create a LabelEncoder object and assign it to le
answer_label= le.fit_transform(df["answer"])

In [34]:
from sklearn.preprocessing import OneHotEncoder
hot_enc = OneHotEncoder()
answer_label= le.fit_transform(df["answer"])

In [35]:
df["answer"] = answer_label

<ipython-input-35-39c574c63442>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["answer"] = answer_label


In [36]:
!pip install datasets

In [44]:
from datasets import Dataset
full_data = Dataset.from_pandas(df)

In [39]:
full_data = full_data.remove_columns('image_name')

# Fine Tuning VILT VQA

In [45]:
from transformers import AutoProcessor, ViltForQuestionAnswering

processor = AutoProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [46]:
df["answer"].unique().shape

(243,)

In [47]:
import torch
from PIL import Image
def pre_process(data):
    homepath = "/content/drive/MyDrive/Colab Notebooks/Visual-Bert-Application/"
    image = Image.open(homepath + data["image_name"]).convert("RGB")
    image = image.resize((448, 448))
    encoding = processor(image, data["question"],padding="max_length",
        truncation=True,return_tensors="pt")
    encoding["labels"] = torch.tensor(data['answer'], dtype=torch.long)
    return encoding

In [48]:
# !pip install pyunpack
# !pip install patool
# !pip install rarfile


In [49]:
#extracting zip from cloud
# from pyunpack import Archive
# Archive('/content/drive/MyDrive/Colab Notebooks/Visual-Bert-Application/Copy of vilt-vqa-rad-finetuned-AIC.rar').extractall('/content/drive/MyDrive/Colab Notebooks/Visual-Bert-Application/Images')

In [50]:
dataset = full_data.map(pre_process)

Map:   0%|          | 0/1124 [00:00<?, ? examples/s]

In [51]:
dataset

Dataset({
    features: ['image_name', 'question', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'],
    num_rows: 1124
})

In [52]:
dataset = dataset.remove_columns(['image_name', 'question', 'answer', 'token_type_ids', 'pixel_mask'])
dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'pixel_values', 'labels'],
    num_rows: 1124
})

In [53]:
dataset = dataset.remove_columns(['attention_mask'])

In [54]:
from transformers import  Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./vilt-vqa-rad-finetuned",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [55]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()

ValueError: too many values to unpack (expected 2)